# 🤖 AI Assistant in Jupyter Notebooks

This notebook demonstrates multiple ways to integrate AI assistants directly into your Jupyter workflow.

## Methods Covered:
1. **Jupyter AI Extension** - Native AI integration with magic commands
2. **Custom AI Assistant Widget** - Interactive chat interface
3. **Code Generation Assistant** - AI-powered code completion
4. **Inline AI Help** - Context-aware assistance


## Setup and Configuration

In [ ]:
# Core imports
import os
import asyncio
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional

# LangChain imports
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# Jupyter and widgets
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
from IPython import get_ipython

# Data and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv()

print("✅ All libraries imported successfully!")

## Method 1: Jupyter AI Extension (Magic Commands)

The Jupyter AI extension provides magic commands that integrate AI directly into your notebook cells.

### Installation Note:
After installing the requirements, you'll need to enable the extension:
```bash
jupyter lab build  # If using JupyterLab
```

In [ ]:
# Configure Jupyter AI for Azure OpenAI
# This would typically be done in the Jupyter AI settings UI
print("Jupyter AI Magic Commands:")
print("%%ai azure-chat-openai:gpt-4o")
print("Explain this code: [your code here]")
print("")
print("%%ai azure-chat-openai:gpt-4o")
print("Generate Python code to create a scatter plot")
print("")
print("Note: You'll need to configure your Azure OpenAI credentials in the Jupyter AI settings.")

## Method 2: Custom AI Assistant Widget

Create a custom interactive AI assistant that lives in your notebook.

In [ ]:
class JupyterAIAssistant:
    def __init__(self):
        # Initialize Azure OpenAI
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
            temperature=0.7,
            max_tokens=1000
        )
        
        # Create UI components
        self.setup_ui()
        self.conversation_history = []
        
    def setup_ui(self):
        # Input components
        self.input_text = widgets.Textarea(
            placeholder='Ask me anything about your code, data, or need help with Python...',
            description='Question:',
            layout=widgets.Layout(width='100%', height='80px')
        )
        
        self.send_btn = widgets.Button(
            description='Ask AI 🤖',
            button_style='primary',
            layout=widgets.Layout(width='120px')
        )
        
        self.clear_btn = widgets.Button(
            description='Clear 🧹',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        
        self.code_help_btn = widgets.Button(
            description='Code Help 💡',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        
        # Output area
        self.output = widgets.Output(
            layout=widgets.Layout(height='400px', overflow='auto')
        )
        
        # Context selector
        self.context_dropdown = widgets.Dropdown(
            options=['General Help', 'Code Debugging', 'Data Analysis', 'Visualization', 'Machine Learning'],
            value='General Help',
            description='Context:',
            layout=widgets.Layout(width='200px')
        )
        
        # Bind events
        self.send_btn.on_click(self.on_send_click)
        self.clear_btn.on_click(self.on_clear_click)
        self.code_help_btn.on_click(self.on_code_help_click)
        
        # Layout
        button_row = widgets.HBox([self.send_btn, self.clear_btn, self.code_help_btn, self.context_dropdown])
        self.widget = widgets.VBox([
            widgets.HTML("<h3>🤖 AI Assistant</h3>"),
            self.input_text,
            button_row,
            self.output
        ])
        
    def get_system_prompt(self, context: str) -> str:
        prompts = {
            'General Help': "You are a helpful AI assistant for Jupyter notebooks. Provide clear, concise answers.",
            'Code Debugging': "You are a Python debugging expert. Help identify and fix code issues. Provide specific solutions.",
            'Data Analysis': "You are a data analysis expert. Help with pandas, numpy, and data manipulation tasks.",
            'Visualization': "You are a data visualization expert. Help create charts with matplotlib, seaborn, and plotly.",
            'Machine Learning': "You are a machine learning expert. Help with scikit-learn, model selection, and ML workflows."
        }
        return prompts.get(context, prompts['General Help'])
        
    def on_send_click(self, b):
        if self.input_text.value.strip():
            self.ask_ai(self.input_text.value, self.context_dropdown.value)
            
    def on_clear_click(self, b):
        self.conversation_history = []
        self.output.clear_output()
        with self.output:
            print("🧹 Conversation cleared!")
            
    def on_code_help_click(self, b):
        # Get the current cell content if available
        help_text = "Analyze the code in the current cell and suggest improvements or explain what it does."
        self.ask_ai(help_text, "Code Debugging")
        
    def ask_ai(self, question: str, context: str = "General Help"):
        with self.output:
            print(f"🧑 You: {question}")
            print("🤖 AI: Thinking...")
            
        try:
            # Prepare messages
            system_prompt = self.get_system_prompt(context)
            messages = [SystemMessage(content=system_prompt)]
            
            # Add conversation history
            messages.extend(self.conversation_history[-6:])  # Keep last 6 messages for context
            messages.append(HumanMessage(content=question))
            
            # Get AI response
            response = self.llm.invoke(messages)
            
            # Update conversation history
            self.conversation_history.extend([
                HumanMessage(content=question),
                AIMessage(content=response.content)
            ])
            
            # Display response
            with self.output:
                clear_output(wait=True)
                print(f"🧑 You: {question}")
                print(f"🤖 AI [{context}]: {response.content}")
                print("-" * 60)
                
        except Exception as e:
            with self.output:
                clear_output(wait=True)
                print(f"🧑 You: {question}")
                print(f"❌ Error: {str(e)}")
                print("Please check your Azure OpenAI configuration.")
                print("-" * 60)
        
        self.input_text.value = ''
        
    def display(self):
        display(self.widget)
        with self.output:
            print("👋 Hi! I'm your AI assistant. Ask me anything about your code or data!")
            print("💡 Try different contexts for specialized help.")
            print("-" * 60)

# Create and display the AI assistant
ai_assistant = JupyterAIAssistant()
ai_assistant.display()

## Method 3: Custom Magic Commands

Create custom magic commands for AI assistance.

In [ ]:
@magics_class
class AIAssistantMagics(Magics):
    
    def __init__(self, shell):
        super().__init__(shell)
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
            temperature=0.3,
            max_tokens=500
        )
    
    @line_magic
    def aihelp(self, line):
        """Get AI help with a question"""
        if not line.strip():
            return "Usage: %aihelp <your question>"
        
        try:
            response = self.llm.invoke([HumanMessage(content=line)])
            display(Markdown(f"**🤖 AI Assistant:**\n\n{response.content}"))
        except Exception as e:
            print(f"❌ Error: {str(e)}")
    
    @cell_magic
    def aicode(self, line, cell):
        """Get AI help with code in the cell"""
        prompt = f"Analyze this code and provide feedback:\n\n```python\n{cell}\n```\n\nProvide suggestions for improvement, explain what it does, or identify any issues."
        
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            display(Markdown(f"**🤖 Code Analysis:**\n\n{response.content}"))
        except Exception as e:
            print(f"❌ Error: {str(e)}")
    
    @line_magic
    def aigenerate(self, line):
        """Generate code based on description"""
        if not line.strip():
            return "Usage: %aigenerate <description of what you want to code>"
        
        prompt = f"Generate Python code for: {line}\n\nProvide clean, well-commented code that can be run in a Jupyter notebook."
        
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            display(Markdown(f"**🤖 Generated Code:**\n\n{response.content}"))
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Register the magic commands
ip = get_ipython()
if ip:
    ip.register_magic_function(AIAssistantMagics(ip).aihelp, 'line')
    ip.register_magic_function(AIAssistantMagics(ip).aicode, 'cell')
    ip.register_magic_function(AIAssistantMagics(ip).aigenerate, 'line')
    print("✅ AI Magic commands registered!")
    print("Available commands:")
    print("  %aihelp <question> - Get AI help")
    print("  %%aicode - Analyze code in cell")
    print("  %aigenerate <description> - Generate code")
else:
    print("❌ Could not register magic commands")

## Method 4: Context-Aware Code Assistant

An AI assistant that understands your notebook context.

In [ ]:
class ContextAwareAssistant:
    def __init__(self):
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
            temperature=0.5
        )
        
    def get_notebook_context(self):
        """Get context about the current notebook state"""
        context = []
        
        # Get IPython instance
        ip = get_ipython()
        if ip:
            # Get user namespace variables
            user_vars = ip.user_ns
            
            # Analyze variables
            for name, obj in user_vars.items():
                if not name.startswith('_') and not callable(obj):
                    obj_type = type(obj).__name__
                    if obj_type in ['DataFrame', 'Series', 'ndarray', 'list', 'dict']:
                        if hasattr(obj, 'shape'):
                            context.append(f"Variable '{name}': {obj_type} with shape {obj.shape}")
                        elif hasattr(obj, '__len__'):
                            context.append(f"Variable '{name}': {obj_type} with length {len(obj)}")
                        else:
                            context.append(f"Variable '{name}': {obj_type}")
        
        return "\n".join(context) if context else "No significant variables found in notebook."
    
    def ask_with_context(self, question: str):
        """Ask AI with notebook context"""
        context = self.get_notebook_context()
        
        prompt = f"""You are an AI assistant helping with a Jupyter notebook. Here's the current context:

NOTEBOOK CONTEXT:
{context}

USER QUESTION:
{question}

Please provide a helpful response considering the notebook's current state."""
        
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    
    def suggest_next_steps(self):
        """Suggest what to do next based on current context"""
        context = self.get_notebook_context()
        
        prompt = f"""Based on this Jupyter notebook context, suggest 3-5 logical next steps:

{context}

Provide specific, actionable suggestions for data analysis, visualization, or modeling."""
        
        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            display(Markdown(f"**🎯 Suggested Next Steps:**\n\n{response.content}"))
        except Exception as e:
            print(f"Error: {str(e)}")

# Create context-aware assistant
context_ai = ContextAwareAssistant()

# Helper functions
def ask_ai_context(question):
    """Quick function to ask AI with context"""
    response = context_ai.ask_with_context(question)
    display(Markdown(f"**🤖 AI Response:**\n\n{response}"))

def suggest_next():
    """Quick function to get next step suggestions"""
    context_ai.suggest_next_steps()

print("✅ Context-aware AI assistant ready!")
print("Use: ask_ai_context('your question') or suggest_next()")

## Testing the Magic Commands

Try these examples:

In [ ]:
# Test the magic commands
%aihelp How do I create a scatter plot in matplotlib?

In [ ]:
%aigenerate Create a function to calculate the mean and standard deviation of a list

In [ ]:
%%aicode
# Example code to analyze
import pandas as pd
data = [1, 2, 3, 4, 5]
df = pd.DataFrame({'values': data})
result = df['values'].mean()

## Testing Context-Aware Assistant

Create some data and test the context-aware features:

In [ ]:
# Create some sample data for testing
import pandas as pd
import numpy as np

# Sample dataset
sample_data = pd.DataFrame({
    'age': np.random.randint(18, 80, 100),
    'income': np.random.normal(50000, 15000, 100),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], 100)
})

# Sample array
scores = np.random.normal(75, 10, 50)

print("✅ Sample data created!")
print(f"DataFrame shape: {sample_data.shape}")
print(f"Scores array shape: {scores.shape}")

In [ ]:
# Test context-aware assistant
ask_ai_context("What kind of analysis can I do with this data?")

In [ ]:
# Get suggestions for next steps
suggest_next()

## 🎉 Summary

You now have multiple ways to integrate AI assistants into your Jupyter notebooks:

### 1. **Interactive Widget Assistant** 🤖
- Full-featured chat interface
- Context-aware responses
- Multiple assistance modes

### 2. **Magic Commands** ✨
- `%aihelp` - Get quick help
- `%%aicode` - Analyze code in cells
- `%aigenerate` - Generate code from descriptions

### 3. **Context-Aware Assistant** 🎯
- Understands your notebook variables
- Suggests relevant next steps
- Provides contextual advice

### 4. **Jupyter AI Extension** (Optional) 🔧
- Native integration with Jupyter
- Built-in magic commands
- UI-based configuration

## Next Steps:
1. Install the updated requirements: `pip install -r requirements.txt`
2. Configure your `.env` file with Azure OpenAI credentials
3. Try each method to see which works best for your workflow
4. Customize the assistants for your specific needs

Happy coding with your AI assistants! 🚀